In [1]:
from bs4 import BeautifulSoup
import requests
import csv, random, time

In [2]:
headers = {
    'authority': 'scrapeme.live',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

In [3]:
URL = 'https://www.tradingview.com/markets/stocks-usa/market-movers-active/'
page = requests.get(URL, headers=headers)

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
results = soup.find_all("tr")   

In [6]:
count = 0
store_ticker = []

# pull tickers for top 100 active stocks

for item in results[1:]:
    text = item.text
    text = text.split("\n")
    store_ticker.append(str(text[5]))
    count += 1

In [7]:
import yfinance as yf
import pygsheets
import pandas as pd
from datetime import datetime

#authorization
gc = pygsheets.authorize(service_file='daily_stocksv2.json')

# open google spreadsheet
sh = gc.open('daily_stocks')

temp = sh.worksheet_by_title('Template') 

# should check if a worksheet for today has already been created or not 
try:
    wks = sh.worksheet_by_title(datetime.now().strftime("%m/%d/%Y"))
except pygsheets.WorksheetNotFound:
    sh.add_worksheet(datetime.now().strftime("%m/%d/%Y"), src_worksheet=temp)
finally:
    wks = sh.worksheet_by_title(datetime.now().strftime("%m/%d/%Y"))
    
i = 2

for s in store_ticker:
    stk = yf.Ticker(s) 
    stk_hist = stk.history(period = '1d', inteval = '5m') # stk_hist is a dataframe columns: symbol,open,high,low,close,volume,dividends,stock splits
    stk_hist.insert(loc=0, column='Symbol', value=s) # add in column for ticker
    stk_hist.drop(stk_hist.columns[[-1,-2]], axis=1, inplace=True) # remove dividend and stock splits columns

    wks.set_dataframe(stk_hist,(i,1), copy_head=False)
    i += 1
    
# google sheets link: https://docs.google.com/spreadsheets/d/1HN3uNLbiI4MjgYqAcct3Vd4VbcxVQviDnbTSjf016lQ/edit#gid=0